In [2]:
!unzip data.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/train_ims/c083238.jpg  
  inflating: data/train_ims/c084872.jpg  
  inflating: data/train_ims/c084f38.jpg  
  inflating: data/train_ims/c086260.jpg  
  inflating: data/train_ims/c087f24.jpg  
  inflating: data/train_ims/c088316.jpg  
  inflating: data/train_ims/c088b71.jpg  
  inflating: data/train_ims/c08a268.jpg  
  inflating: data/train_ims/c08a54c.jpg  
  inflating: data/train_ims/c08b9ee.jpg  
  inflating: data/train_ims/c08cf55.jpg  
  inflating: data/train_ims/c08d23b.jpg  
  inflating: data/train_ims/c08e144.jpg  
  inflating: data/train_ims/c08e609.jpg  
  inflating: data/train_ims/c08edac.jpg  
  inflating: data/train_ims/c08ef54.jpg  
  inflating: data/train_ims/c08f158.jpg  
  inflating: data/train_ims/c08f2fb.jpg  
  inflating: data/train_ims/c093473.jpg  
  inflating: data/train_ims/c094ce8.jpg  
  inflating: data/train_ims/c0963cd.jpg  
  inflating: data/train_ims/c099962.jpg  
  inflating: data/train_i

In [ ]:
import pandas as pd
import numpy as np
import cv2
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.transform import resize
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.calibration import CalibratedClassifierCV

In [4]:
import pandas as pd
import os

# Paths
train_csv = 'data/train.csv'
test_csv = 'data/test.csv'
train_images_path = 'data/train_ims/'
test_images_path = 'data/test_ims/'

# Load CSV
train_df = pd.read_csv(train_csv)
print(train_df.head())



       im_name  label
0  00016cd.jpg      6
1  0001808.jpg      2
2  0002399.jpg      1
3  0003973.jpg      3
4  00061cc.jpg      4


In [5]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.transform import resize

def load_and_preprocess_images(df, folder):
    images = []
    labels = []
    for index, row in df.iterrows():
        image_path = f"{folder}/{row['im_name']}"
        image = imread(image_path)
        images.append(image)
        labels.append(row['label'])
    return np.array(images), np.array(labels)

train_data, train_labels = load_and_preprocess_images(train_df, train_images_path)


In [6]:
test_df = pd.read_csv(test_csv)
test_data, test_labels = load_and_preprocess_images(test_df, test_images_path)


In [ ]:
! pip install cupy-cuda110

ERROR: Could not find a version that satisfies the requirement cupy-cuda112 (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\OWNER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for cupy-cuda112


In [12]:
from sklearn.base import BaseEstimator, TransformerMixin
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.transform import resize
import cupy as cp

class HOGFeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, resize_shape=(64, 64), orientations=12, pixels_per_cell=(8, 8), cells_per_block=(4, 4)):
        self.resize_shape = resize_shape
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        hog_features = []
        for img in X:
            if img.ndim == 3:
                img_cp = cp.array(img)
                img_gray = 0.2989 * img_cp[:, :, 0] + 0.5870 * img_cp[:, :, 1] + 0.1140 * img_cp[:, :, 2]
                img_gray = cp.asnumpy(img_gray)
            else:
                img_gray = img
            if self.resize_shape:
                img_resized = resize(img_gray, self.resize_shape, anti_aliasing=True)
            # Extract HOG features
            features = hog(
                img_resized,
                orientations=self.orientations,
                pixels_per_cell=self.pixels_per_cell,
                cells_per_block=self.cells_per_block,
                block_norm='L2-Hys',
                visualize=False
            ).flatten()
            hog_features.append(features)
        return np.array(hog_features)



C:\Users\OWNER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [13]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import cupy as cp
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('hog', HOGFeatureExtractor()),  # Custom HOG feature extractor
    ('scaler', StandardScaler()),  # Standardize the features
    ('pca', PCA(n_components=256)),  # Apply PCA for dimensionality reduction
    ('svm', SVC(kernel='rbf', C=10, gamma='auto'))  # SVM classifier
])


# Train the classifier
pipeline.fit(train_data, train_labels)

# Make predictions on the test set
test_predictions = pipeline.predict(test_data)

test_df['label'] = test_predictions
test_df.to_csv('updated_test_predictions.csv', index=False)
print("Predictions saved to 'updated_test_predictions.csv'")

RuntimeError: CuPy failed to load nvrtc64_112_0.dll: FileNotFoundError: Could not find module 'nvrtc64_112_0.dll' (or one of its dependencies). Try using the full path with constructor syntax.